In [1]:
import pandas as pd
import numpy as np
import sklearn
import pickle 
import time 
import datetime
import warnings
import matplotlib.pyplot as plt
import torch
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import graft
import torch_geometric


In [2]:
with open('./fraudTrain.pkl', 'rb') as file:
    fraudTrain = pickle.load(file)    

In [3]:
df_test = pd.read_csv('~/Dropbox/Data/df_test.csv')

In [4]:
df = df_test[df_test['amt'] <= 80]

In [5]:
cc_amt_80 = list(set(df['cc_num']))

In [6]:
df_fraud = df[df.is_fraud == 1]
df_fraud_cc = list(set(df_fraud['cc_num']))

In [7]:
df2 = fraudTrain[fraudTrain['cc_num'].isin(df_fraud_cc)]

In [8]:
df2.shape

(311504, 22)

In [9]:

df3 = fraudTrain[fraudTrain['cc_num'].isin(df_fraud_cc)].reset_index()
df_group = df3.groupby("cc_num")["is_fraud"].agg(['count', 'sum']).reset_index()
df_group.columns = ['cc_num', 'total_transactions', 'fraud_transactions']
first = df3.groupby('cc_num')['first'].first().reset_index() 
last = df3.groupby('cc_num')['last'].first().reset_index()
name = first['first'] + ' ' + last['last']
df_group['name'] = name
df_group.insert(1, 'name', df_group.pop('name'))
df_group.sort_values(by=['total_transactions', 'fraud_transactions'], ascending=[True, True])

,cc_num,name,total_transactions,fraud_transactions
204,4.734310e+15,Mark Pennington,7,7
63,3.881750e+13,Molly Marshall,8,8
185,4.157210e+15,Bryan Gill,8,8
260,4.481170e+18,Jennifer Johnson,8,8
109,2.287280e+15,Calvin Molina,9,9
...,...,...,...,...
67,1.800650e+14,Krystal Gamble,2500,20
66,1.800480e+14,Mary Wall,2513,14
7,5.713650e+11,Barbara Taylor,2521,10
226,6.011440e+15,Allison Allen,2529,14


In [10]:
df_group.to_csv('df_group3.csv', index=False)

In [11]:
def compute_time_difference(group):
    n = len(group)
    result = []
    for i in range(n):
        for j in range(n):
            time_difference = abs((group.iloc[i].trans_date_trans_time - group.iloc[j].trans_date_trans_time).total_seconds())
            result.append([group.index[i], group.index[j], time_difference])
    return result

In [ ]:
all_graphs = []

unique_cc_nums = df3['cc_num'].unique()
for c_sample in unique_cc_nums:
    groups = df3[df3['cc_num'] == c_sample]
    
    if len(groups) < 2:
        continue  # Skip if there are not enough transactions to form edges
    
    edge_index = compute_time_difference(groups)
    edge_index = np.array(edge_index).astype(np.float64)
    edge_index = edge_index[edge_index[:, 0] != edge_index[:, 1]]
    
    if len(edge_index) == 0:
        continue  # Skip if no valid edges were formed
    
    theta = 1e7
    edge_index[:, 2] = (np.exp(-edge_index[:, 2] / theta) != 1) * (np.exp(-edge_index[:, 2] / theta))
    edge_index = edge_index.tolist()
    
    selected_indices = groups.index
    is_fraud_values = df3.loc[selected_indices, 'is_fraud'].tolist()
    
    unique_nodes = set(np.array(edge_index)[:, :2].reshape(-1).tolist())
    dict_map = {n: i for i, n in enumerate(unique_nodes)}
    
    source, target = np.array(edge_index)[:, 0], np.array(edge_index)[:, 1]
    edge_index_selected = torch.tensor([[dict_map[s] for s in source.tolist()], [dict_map[t] for t in target.tolist()]])
    
    graph = torch_geometric.data.Data(
        edge_index=edge_index_selected,
        edge_attr=torch.tensor(np.array(edge_index)[:, -1], dtype=torch.float),
        x=torch.tensor(groups['amt'].values, dtype=torch.float).unsqueeze(1),  # Assuming 'amt' is the feature
        y=torch.tensor(groups['is_fraud'].values, dtype=torch.long)
    )

    graph.cc_num = c_sample
    all_graphs.append(graph)

In [ ]:
with open('all_graphs10.pkl', 'wb') as f:
    pickle.dump(all_graphs, f)

In [ ]:

dr_opts = { 
    'output_size': (500,500),
    'edge_marker_size': 1,
    #'output':"ten-nodes.pdf",
}

In [ ]:
graph_number = 100

In [ ]:

graft.graph.plot_undirected_weighted(
    all_graphs[graph_number],
    node_color=all_graphs[graph_number].y,
    node_size=all_graphs[graph_number].x,
    edge_weight_text=False,
    draw_options= dr_opts,
)
all_graphs[graph_number].cc_num

---